# Comentario: Aquí se importan las bibliotecas necesarias y se establece la ruta de exportación para los archivos. Luego, se crea la carpeta si no existe.

In [1]:
# Importación de bibliotecas necesarias
import csv
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

# Ruta de exportación de archivos
ruta_exportacion = r'C:\resultados'

# Crear la carpeta si no existe
if not os.path.exists(ruta_exportacion):
    os.makedirs(ruta_exportacion)


# Esta función toma el código HTML de un trayecto y extrae la información de precios. Se asume que hay una estructura específica de clases en el HTML y por eso usamos BeautifulSoup.

In [2]:
def extraer_precios(html_code):
    try:
        soup = BeautifulSoup(html_code, 'html.parser')

        # Encuentra el elemento que contiene la información de precios
        precios_element = soup.find('div', class_='tu-clase-de-precios')  # Reemplaza 'tu-clase-de-precios' con la clase correcta

        # Extrae los precios
        precios = []
        if precios_element:
            precios_items = precios_element.find_all('span', class_='tu-clase-de-precio')  # Reemplaza 'tu-clase-de-precio' con la clase correcta
            precios = [precio.get_text(strip=True) for precio in precios_items]

        return precios

    except Exception as e:
        print(f"Error al extraer precios: {str(e)}")
        return []


# Esta función extrae las fechas de IDA y VUELTA de una URL codificada.

In [3]:
def obtener_fecha_desde_url(url):
    try:
        # Parsea la URL para encontrar las fechas
        partes_url = url.split('%2C')
        fecha_ida = partes_url[-2]
        fecha_vuelta = partes_url[-1]
        return fecha_ida, fecha_vuelta
    except Exception as e:
        print(f"Error al obtener fechas desde la URL: {str(e)}")
        return None, None


# Esta función toma un elemento de trayecto, tipo de trayecto, precios, número de viaje y fechas, y devuelve un diccionario con la información del viaje.

In [4]:
def obtener_info_viaje(trayecto_element, trayecto_tipo, precios, numero_viaje, fecha_ida, fecha_vuelta):
    try:
        trayecto_html = trayecto_element.get_attribute("outerHTML")
        soup = BeautifulSoup(trayecto_html, 'html.parser')

        hora_salida = soup.find('span', class_='sc-dLMFU gVWdyu JourneyTime__HourFont-sc-1ej0v4k-1 cBmPEj').get_text(strip=True)
        ciudad_origen = soup.find('span', class_='sc-dLMFU gVWdyu JourneyTime__CityFont-sc-1ej0v4k-2 jpapGu').get_text(strip=True)

        llegada_info_element = soup.find('div', class_='JourneyTime__TimeContainer-sc-1ej0v4k-0 bcTRis')
        hora_llegada = llegada_info_element.find('span', class_='sc-dLMFU gVWdyu JourneyTime__HourFont-sc-1ej0v4k-1 cBmPEj').get_text(strip=True)
        ciudad_destino = llegada_info_element.find('span', class_='sc-dLMFU gVWdyu JourneyTime__CityFont-sc-1ej0v4k-2 jpapGu').get_text(strip=True)

        print("=" * 35)
        print(f'{"Viaje " + str(numero_viaje):^35}')
        print(f'{"- Trayecto " + trayecto_tipo.upper() + ":":^35}')
        print(f"Ciudad de origen: {ciudad_origen} -> Ciudad de destino: {ciudad_destino}")
        print(f"Hora de salida: {hora_salida} -> Hora de llegada: {hora_llegada}")

        precios_extraidos = extraer_precios(trayecto_html)
        for i, precio_info in enumerate(precios_extraidos):
            print(f"===================================")
            print(f'{"              Precio " + str(i+1) + ":":^35}')
            print(f'{"PrecioMonto:"} {precio_info["PrecioMonto"]}')
            print(f'{"PrecioDecimales:"} {precio_info["PrecioDecimales"]}')
            print(f'{"PrecioDivisa:"} {precio_info["PrecioDivisa"]}')
            print("=" * 35)

        precios.extend(precios_extraidos)

        info_viaje = {
            "Número de Viaje": numero_viaje,
            "Tipo de Trayecto": trayecto_tipo,
            "Ciudad de Origen": ciudad_origen,
            "Ciudad de Destino": ciudad_destino,
            "Hora de Salida": hora_salida,
            "Hora de Llegada": hora_llegada,
            "Fecha IDA": fecha_ida,
            "Fecha VUELTA": fecha_vuelta,
            "Precios": precios_extraidos
        }

        return info_viaje

    except Exception as e:
        print(f"Error al obtener información del trayecto: {str(e)}")
        return None


# En este bloque principal, se configura el navegador, se procesan las dos URLs, y se exportan los resultados a archivos CSV, JSON y Excel. También, se manejan excepciones durante el proceso.

In [5]:
def procesar_url(url, driver):
    driver.get(url)

    # Espera hasta que el botón de aceptar cookies esté presente
    accept_cookies_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "CybotCookiebotDialogBodyButtonAccept"))
    )
    accept_cookies_button.click()

    # Espera hasta que al menos un trayecto esté presente
    trayectos = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="styles__Journey-sc-1ornisu-4 gOhLrT"]'))
    )

    # Lista para almacenar información de viajes y precios
    lista_info_viajes = []
    lista_precios = []

    # Obtiene las fechas desde la URL
    fecha_ida, fecha_vuelta = obtener_fecha_desde_url(url)

    # Itera sobre los elementos de trayecto y llama a la función obtener_info_viaje intercalando IDA y VUELTA
    for i, trayecto_element in enumerate(trayectos):
        numero_viaje = i // 2 + 1  # Calcula el número de viaje (división entera)
        if i % 2 == 0:
            info_viaje = obtener_info_viaje(trayecto_element, "IDA", lista_precios, numero_viaje, fecha_ida, fecha_vuelta)
            if info_viaje:
                lista_info_viajes.append(info_viaje)
        else:
            info_viaje = obtener_info_viaje(trayecto_element, "VUELTA", lista_precios, numero_viaje, fecha_ida, fecha_vuelta)
            if info_viaje:
                lista_info_viajes[-1]["Precios"].extend(info_viaje["Precios"])

    return lista_info_viajes

# Configuración del navegador (en este caso, se utiliza Firefox con GeckoDriver)
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("-private")  # Habilita el modo incógnito
driver = webdriver.Firefox(options=firefox_options)

try:
    # Procesa la primera URL
    url_1 = "https://www.atrapalo.com/ms/results?journeys=VGO%2CMAD%2C20240228%7CMAD%2CVGO%2C20240229"
    resultados_1 = procesar_url(url_1, driver)

    # Procesa la segunda URL
    url_2 = "https://www.atrapalo.com/ms/results?journeys=VGO%2CMAD%2C20240229%7CMAD%2CVGO%2C20240301"
    resultados_2 = procesar_url(url_2, driver)

    # Exportar datos a CSV
    csv_file_path = os.path.join(ruta_exportacion, "resultadoAtrapaloSelenium.csv")
    with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=resultados_1[0].keys())
        csv_writer.writeheader()
        csv_writer.writerows(resultados_1 + resultados_2)

    # Exportar datos a JSON
    json_file_path = os.path.join(ruta_exportacion, "resultadoAtrapaloSelenium.json")
    with open(json_file_path, "w", encoding="utf-8") as json_file:
        json.dump(resultados_1 + resultados_2, json_file, ensure_ascii=False, indent=4)

    # Exportar datos a XLS (Excel)
    excel_file_path = os.path.join(ruta_exportacion, "resultadoAtrapaloSelenium.xlsx")
    df = pd.DataFrame(resultados_1 + resultados_2)
    df.to_excel(excel_file_path, index=False)

    print("Archivos exportados exitosamente.")
except Exception as export_error:
    print(f"Error al exportar archivos: {str(export_error)}")
finally:
    # Cierra el navegador
    driver.quit()


              Viaje 1              
          - Trayecto IDA:          
Ciudad de origen: VGO -> Ciudad de destino: MAD
Hora de salida: 09:20 -> Hora de llegada: 10:35
              Viaje 1              
        - Trayecto VUELTA:         
Ciudad de origen: MAD -> Ciudad de destino: VGO
Hora de salida: 07:20 -> Hora de llegada: 08:40
              Viaje 2              
          - Trayecto IDA:          
Ciudad de origen: VGO -> Ciudad de destino: MAD
Hora de salida: 06:30 -> Hora de llegada: 07:50
              Viaje 2              
        - Trayecto VUELTA:         
Ciudad de origen: MAD -> Ciudad de destino: VGO
Hora de salida: 07:20 -> Hora de llegada: 08:40
              Viaje 3              
          - Trayecto IDA:          
Ciudad de origen: VGO -> Ciudad de destino: MAD
Hora de salida: 17:40 -> Hora de llegada: 19:00
              Viaje 3              
        - Trayecto VUELTA:         
Ciudad de origen: MAD -> Ciudad de destino: VGO
Hora de salida: 07:20 -> Hora de llegada